In [1]:
import pandas as pd

In [2]:
INPUT_CSV_FILES = [
#     'predictions/predictions-c.csv',
#     'predictions/predictions-d2.csv',
#     'predictions/predictions-e@17.csv',
    'predictions/predictions-f@13.csv',
]

OUT_FILE = 'submissions/s-10.csv'

ENSEMBLING_METHOD = 'mean' # mean/voting

In [3]:
%run 'lib.ipynb'
%run 'data-generator.ipynb'

In [4]:
dfs = [pd.DataFrame.from_csv(f) for f in INPUT_CSV_FILES]

In [5]:
predictions = {} # file: [pq, p2,...]

for df in dfs:
    for file, p in df.iterrows():
        if not file in predictions:
            predictions[file] = []
        predictions[file].append(list(p))

In [6]:
submission = {}
mean_voting_disagrees = 0

for file, preds in predictions.items():

    label_i_mean = np.argmax(np.mean(preds, axis=0))
    label_i_voting = np.argmax(np.bincount(np.argmax(preds, axis=1)))
    if label_i_mean != label_i_voting: mean_voting_disagrees += 1

    if ENSEMBLING_METHOD == 'mean':
        submission[file] = LABELS[label_i_mean]
    elif ENSEMBLING_METHOD == 'voting':
        submission[file] = LABELS[label_i_voting]

print('mean_voting_disagrees', mean_voting_disagrees)

mean_voting_disagrees 0


In [7]:
submission_df = pd.DataFrame.from_dict(submission, orient='index')
submission_df.columns = ['label']
submission_df.to_csv(OUT_FILE, index_label='fname')